# CNN for Image Classification
This notebook builds a convolutional neural network (CNN) for classifying images. It classifies images of cats and dogs. The dataset consists of 4000 images of cats and dogs with 2000 images for each animal.It uses the images from the well known Kaggle dataset which can be found here, https://www.kaggle.com/c/dogs-vs-cats/data.

# Download the data

In [1]:
# import packages for directory creating and for copying image files
import os, shutil

In [2]:
# path of downloaded dataset
original_dataset_dir = "/home/moshiur/Downloads/dogs-vs-cats/train"
# the directory where the dataset is stored for analysis
base_dir = "/home/moshiur/Research/Machine_Learning/ML_practice/Chollet_DL_Python/datasets/cats_and_dogs_small"
os.mkdir(base_dir)

In [3]:
# directories for training, validation and test data
train_dir = os.path.join(base_dir, "train")
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, "validation")
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, "test")
os.mkdir(test_dir)

In [5]:
# directories for training cat and dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

In [6]:
# directories for validation cat and dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

In [7]:
# directories for test cat and dog pictures
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [10]:
# copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

In [11]:
# copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)

In [12]:
# copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)

In [13]:
# copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [14]:
# copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [15]:
# copy next 500 dog images to test_cats_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [19]:
# check the copy of images
print('Total training cat images: ', len(os.listdir(train_cats_dir)))
print('Total training dog images: ', len(os.listdir(train_dogs_dir)))
print('Total validation cat images: ', len(os.listdir(validation_cats_dir)))
print('Total test dog images: ', len(os.listdir(test_dogs_dir)))

Total training cat images:  1000
Total training dog images:  1000
Total validation cat images:  500
Total test dog images:  500


# Build the CNN

In [20]:
# imptensorflow.kerasd layers from keras (using Tensorflow 2.0)
from tensorflow.keras import models
from tensorflow.keras import layers

In [21]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2, 2)))
# flatten the output of the MaxPool layer
model.add(layers.Flatten())
# a densely connected layer of 512 neurons
model.add(layers.Dense(512, activation='relu'))
# single output neuron as the output of the network is binary i.e., is the image is of a dog or a cat?
model.add(layers.Dense(1, activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36992)             0

In [23]:
# compile the model
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

# Data preprocessing

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# scale all images by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# training data (for training images) generator
train_generator = train_datagen.flow_from_directory(
        # target dir
        train_dir,
        # all images will be of size (150, 150)
        target_size = (150, 150),
        #batch size for images
        batch_size = 20,
        # binary labels since we use binary_crossentropy as the cost function
        class_mode = 'binary'
)

# validation data (for validation images) generator
validation_generator = test_datagen.flow_from_directory(
        # target dir
        validation_dir,
        # all images will be of size (150, 150)
        target_size = (150, 150),
        #batch size for images
        batch_size = 20,
        # binary labels since we use binary_crossentropy as the cost function
        class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [25]:
# let's have a look at the output of the train_generator
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [26]:
# fit the generator to the model
history = model.fit_generator(
    train_generator, steps_per_epoch=100, epochs=30,
    validation_data = validation_generator, validation_steps=20)

Epoch 1/30
100/100 [==============================] - 210s 2s/step - loss: 0.7151 - acc: 0.5345 - val_loss: 0.6543 - val_acc: 0.6400
Epoch 2/30
100/100 [==============================] - 113s 1s/step - loss: 0.6368 - acc: 0.6380 - val_loss: 0.6094 - val_acc: 0.6750
Epoch 3/30
100/100 [==============================] - 109s 1s/step - loss: 0.5787 - acc: 0.6925 - val_loss: 0.5869 - val_acc: 0.6875
Epoch 4/30
100/100 [==============================] - 109s 1s/step - loss: 0.5223 - acc: 0.7480 - val_loss: 0.6444 - val_acc: 0.6275
Epoch 5/30
100/100 [==============================] - 109s 1s/step - loss: 0.4883 - acc: 0.7635 - val_loss: 0.5824 - val_acc: 0.7000
Epoch 6/30
100/100 [==============================] - 111s 1s/step - loss: 0.4510 - acc: 0.7860 - val_loss: 0.5555 - val_acc: 0.7050
Epoch 7/30
100/100 [==============================] - 113s 1s/step - loss: 0.4083 - acc: 0.8275 - val_loss: 0.6202 - val_acc: 0.6975
Epoch 8/30
100/100 [==============================] - 113s 1s/step - 

In [27]:
import matplotlib.pyplot as plt

In [31]:
plt.imshow(train_cats_dir/cat/cat.1.jpg)

SyntaxError: invalid syntax (<ipython-input-31-3c26a2243cef>, line 1)